In [ ]:
from .qwen2_vl_encoder_pixel_classification import Qwen2VLTForMalformPixelDetectionV1 
from transformers.models.qwen2_vl.image_processing_qwen2_vl import Qwen2VLImageProcessor

VIS_MODEL_PATH = "./Qwen2-VL-7B-Instruct-Config"
model = Qwen2VLTForMalformPixelDetectionV1(2, VIS_MODEL_PATH)
image_processor = Qwen2VLImageProcessor.from_pretrained(VIS_MODEL_PATH)

In [ ]:
from safetensors.torch import load_file

safe_tensor_path = "./model.safetensors"
state_dict = load_file(safe_tensor_path)

model.load_state_dict(state_dict)
model.eval()
model.to("cuda")

In [ ]:
from PIL import Image
from tqdm import tqdm
import torch

image_path = "./example1.png"


img = Image.open(image_path)
img = img.resize((560, 560), Image.LANCZOS)

image_input = image_processor(img, return_tensors="pt").to("cuda")
pixel_values = image_input['pixel_values'].unsqueeze(0)
image_grid_thw = image_input['image_grid_thw']
logits = model(pixel_values, image_grid_thw).logits
prob = torch.argmax(logits.softmax(dim=-1),dim=-1)[0].cpu().numpy()

alpha = 0.4
img_array = np.array(img)
red_mask = np.zeros_like(img_array)
red_mask[pixel_array >= 0.5] = [255, 0, 0]

overlayed_img = img_array.copy()
overlayed_img = (1 - alpha) * img_array + alpha * red_mask
overlayed_img = overlayed_img.astype(np.uint8)
overlayed_img = Image.fromarray(overlayed_img)

draw = ImageDraw.Draw(overlayed_img)
overlayed_img.show()